In [9]:
import sys
import os
import json
from google.protobuf.json_format import Parse
sys.path.insert(0, 'compiled_protobufs')
from taskmap_pb2 import TaskMap
from pyserini.search import LuceneSearcher
import csv
from pprint import pprint
import pandas as pd

In [7]:
searcher_config = {
    'BM25': {'b': 0.4, 'k1': 0.9},
    # 'BM25+RM3':{
    #     'BM25': {'b': 0.4, 'k1': 0.9},
    #     'RM3': {'fb_terms': 10, 'fb_docs': 10, 'original_query_weight': 0.5},
    # }
}

In [26]:
taskmap_cooking_index_path = os.path.join(os.getcwd(), "indexes", "food", "system_index_sparse")

def get_pyserini_docs(searcher, query, k, query_id):
    """ Print pyserini results based on specific query, searcher, and k. """
    results = []
    for hit in searcher.search(q=query, k=k):
        doc_string = hit.raw
        doc_json = json.loads(doc_string)
        taskmap_json = doc_json['recipe_document_json']
        taskmap = Parse(json.dumps(taskmap_json), TaskMap())
        result = {
            "doc-id" : taskmap.taskmap_id, 
            "doc-title" : taskmap.title, 
            "doc-url" : taskmap.source_url, 
            "score": round(float(hit.score),3),
            "query-id": query_id,
            "query": query,
            "taskgraph" : taskmap,
        }
        results.append(result)
    return results

def get_dict_keys_list(judgments, fieldnames):
    new_list = []
    for judgment in judgments:
        dict_item = {}
        for key in fieldnames:
            print(key)
            dict_item[key] = judgment[key]
        new_list.append(dict_item)
    return new_list
    

In [21]:
annotations_path = os.path.join(os.getcwd(), "datasets", "judgments", "cooking-annotations2.csv")
annotation_pairs = os.path.join(os.getcwd(), "datasets", "judgments", "annotation-pairs.csv")

queries_path = os.path.join(os.getcwd(), "datasets", "queries", "cooking_queries.csv")
cooking_queries = pd.read_csv(queries_path).iloc[:10]

In [28]:
searcher = LuceneSearcher(index_dir=taskmap_cooking_index_path)
searcher.set_bm25(b=0.4,k1=0.9)

new_judgments = []
for idx, query in cooking_queries.iterrows():
    new_judgments += get_pyserini_docs(searcher, query["target query"], k=5, query_id=idx)
for idx, judgement in enumerate(new_judgments):
    judgement["judgment-id"] = idx

In [32]:
with open(annotation_pairs, 'w') as csvfile:
    fieldnames = ["judgment-id", "query", "query-id", "taskgraph"]
    print(fieldnames)
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(get_dict_keys_list(new_judgments, fieldnames))

with open(annotations_path, 'w') as csvfile:
    fieldnames = ['judgment-id', 'doc-id', "query-id", "relevance", "justification"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(get_dict_keys_list(new_judgments, fieldnames[:-2]))


['judgment-id', 'query', 'query-id', 'taskgraph']
judgment-id
query
query-id
taskgraph
judgment-id
query
query-id
taskgraph
judgment-id
query
query-id
taskgraph
judgment-id
query
query-id
taskgraph
judgment-id
query
query-id
taskgraph
judgment-id
query
query-id
taskgraph
judgment-id
query
query-id
taskgraph
judgment-id
query
query-id
taskgraph
judgment-id
query
query-id
taskgraph
judgment-id
query
query-id
taskgraph
judgment-id
query
query-id
taskgraph
judgment-id
query
query-id
taskgraph
judgment-id
query
query-id
taskgraph
judgment-id
query
query-id
taskgraph
judgment-id
query
query-id
taskgraph
judgment-id
query
query-id
taskgraph
judgment-id
query
query-id
taskgraph
judgment-id
query
query-id
taskgraph
judgment-id
query
query-id
taskgraph
judgment-id
query
query-id
taskgraph
judgment-id
query
query-id
taskgraph
judgment-id
query
query-id
taskgraph
judgment-id
query
query-id
taskgraph
judgment-id
query
query-id
taskgraph
judgment-id
query
query-id
taskgraph
judgment-id
query
query-i